In [224]:
import datetime
from datetime import date
from datetime import datetime
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
import altair as alt
import time
import itertools
import numpy as np
import operator
from datetime import timedelta
from ipywidgets import interact, IntSlider
from IPython.display import display
import sklearn
import numpy as np
from sklearn.linear_model import LogisticRegression
# import seaborn as sns
# from matplotlib import pyplot as plt
# %matplotlib inline
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.pipeline import make_pipeline as make_pipeline_with_sampler
from imblearn.under_sampling import RandomUnderSampler

from pycaret.classification import * 

# pd.options.display.max_columns = 5000
# pd.options.display.max_rows = 5000

In [225]:
#Loading Dataset
dataset = pd.read_parquet('dataframes_blog/dataset_for_model_final.parquet') 

In [226]:
dataset = dataset[dataset['drop'] == 0]

In [227]:
#Loading Models
week_model = load_model('model_1_week_tuned_final')
month_model = load_model('model_1_month_final')
quarter_model = load_model('model_1_quarter_tuned_final')
semester_model = load_model('model_1_semester_tuned_final')
year_model = load_model('model_1_year_tuned_final')

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [228]:
# Cleaning NaNs

dataset_no_nans = dataset.fillna(0)

In [229]:
# Choosing the features we are going to use

dataset_to_predict = dataset_no_nans[['Height', 'Weight', 'defender', 'attacker', 'midfielder', 'goalkeeper', 'right_foot', 'age', 'cum_injury_total', \
    'weeks_since_last_injury', 'Min_cum', 'Gls_cum', 'Ast_cum', 'PK_cum', 'PKatt_cum', 'Sh_cum', 'SoT_cum', 'CrdY_cum', 'CrdR_cum', \
    'Touches_cum', 'Press_cum', 'Tkl_cum', 'Int_cum', 'Blocks_cum', 'xG_cum', 'npxG_cum', 'xA_cum', 'SCA_cum', 'GCA_cum', 'Cmp_cum', 'Att_cum', \
    'Prog_cum', 'Carries_cum', 'Prog.1_cum', 'Succ_cum', 'Att.1_cum', 'Fls_cum', 'Fld_cum', 'Off_cum', 'Crs_cum', 'TklW_cum', 'OG_cum', \
    'PKwon_cum', 'PKcon_cum', 'Serie A_cum', 'Premier League_cum', 'La Liga_cum', 'Ligue 1_cum', 'Bundesliga_cum', 'Champions Lg_cum', \
    'Europa Lg_cum', 'FIFA World Cup_cum', 'UEFA Nations League_cum', 'UEFA Euro_cum', 'Copa América_cum', 'Away_cum', 'Home_cum', \
    'Neutral_cum']]

In [230]:
#Using saved models to predict injuries

predicted_injuries_1_week = week_model[23].predict(dataset_to_predict)
predicted_injuries_1_month = month_model[23].predict(dataset_to_predict)
predicted_injuries_1_quarter = quarter_model[23].predict(dataset_to_predict)
predicted_injuries_1_semester = semester_model[23].predict(dataset_to_predict)
predicted_injuries_1_year = year_model[23].predict(dataset_to_predict)

In [231]:
# Creating predictions_df to include the whole dataset plus predicted injuries and actual injures

predictions_df = dataset_to_predict
predictions_df['predicted_injuries_1_week'] = predicted_injuries_1_week
predictions_df['predicted_injuries_4_week'] = predicted_injuries_1_month
predictions_df['predicted_injuries_12_week'] = predicted_injuries_1_quarter
predictions_df['predicted_injuries_26_week'] = predicted_injuries_1_semester
predictions_df['predicted_injuries_52_week'] = predicted_injuries_1_year

In [232]:
# Merging the complete dataset with the predictions of the model

dataset_with_prediction = pd.merge(dataset, predictions_df[['predicted_injuries_1_week', 'predicted_injuries_4_week', \
    'predicted_injuries_12_week', 'predicted_injuries_26_week', 'predicted_injuries_52_week']], left_index=True, right_index=True)

In [233]:
# Picking the columns we are going to keep

dataset_with_prediction = dataset_with_prediction[['FBRefID', 'name', 'date', 'Min', 'Position:', 'cum_week', 'defender', 'attacker', 'midfielder', 'goalkeeper', \
    'cum_injury_in_1_week', 'cum_injury_in_4_week',  'cum_injury_in_12_week', 'cum_injury_in_26_week', 'cum_injury_in_52_week', 'Min_cum', \
    'predicted_injuries_1_week', 'predicted_injuries_4_week', 'predicted_injuries_12_week', 'predicted_injuries_26_week', \
    'predicted_injuries_52_week']]

In [234]:
# Creating unique_predicted_injuries to filter out repeated injuries

dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_week'] == 1) & (dataset_with_prediction['predicted_injuries_1_week']. \
    shift(1) == 1), 'unique_predicted_injuries_1_week'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_week'] == 1) & (dataset_with_prediction['predicted_injuries_1_week']. \
    shift(1) == 0), 'unique_predicted_injuries_1_week'] = 1
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_week'] == 0) & (dataset_with_prediction['predicted_injuries_1_week']. \
    shift(1) == 0), 'unique_predicted_injuries_1_week'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_1_week'] == 0) & (dataset_with_prediction['predicted_injuries_1_week']. \
    shift(1) == 1), 'unique_predicted_injuries_1_week'] = 0

dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_4_week'] == 1) & (dataset_with_prediction['predicted_injuries_4_week']. \
    shift(1) == 1), 'unique_predicted_injuries_4_week'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_4_week'] == 1) & (dataset_with_prediction['predicted_injuries_4_week']. \
    shift(1) == 0), 'unique_predicted_injuries_4_week'] = 1
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_4_week'] == 0) & (dataset_with_prediction['predicted_injuries_4_week']. \
    shift(1) == 0), 'unique_predicted_injuries_4_week'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_4_week'] == 0) & (dataset_with_prediction['predicted_injuries_4_week']. \
    shift(1) == 1), 'unique_predicted_injuries_4_week'] = 0

dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_12_week'] == 1) & (dataset_with_prediction['predicted_injuries_12_week']. \
    shift(1) == 1), 'unique_predicted_injuries_12_week'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_12_week'] == 1) & (dataset_with_prediction['predicted_injuries_12_week']. \
    shift(1) == 0), 'unique_predicted_injuries_12_week'] = 1
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_12_week'] == 0) & (dataset_with_prediction['predicted_injuries_12_week']. \
    shift(1) == 0), 'unique_predicted_injuries_12_week'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_12_week'] == 0) & (dataset_with_prediction['predicted_injuries_12_week']. \
    shift(1) == 1), 'unique_predicted_injuries_12_week'] = 0

dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_26_week'] == 1) & (dataset_with_prediction['predicted_injuries_26_week']. \
    shift(1) == 1), 'unique_predicted_injuries_26_'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_26_week'] == 1) & (dataset_with_prediction['predicted_injuries_26_week']. \
    shift(1) == 0), 'unique_predicted_injuries_26_week'] = 1
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_26_week'] == 0) & (dataset_with_prediction['predicted_injuries_26_week']. \
    shift(1) == 0), 'unique_predicted_injuries_26_week'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_26_week'] == 0) & (dataset_with_prediction['predicted_injuries_26_week']. \
    shift(1) == 1), 'unique_predicted_injuries_26_week'] = 0

dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_52_week'] == 1) & (dataset_with_prediction['predicted_injuries_52_week']. \
    shift(1) == 1), 'unique_predicted_injuries_52_week'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_52_week'] == 1) & (dataset_with_prediction['predicted_injuries_52_week']. \
    shift(1) == 0), 'unique_predicted_injuries_52_week'] = 1
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_52_week'] == 0) & (dataset_with_prediction['predicted_injuries_52_week']. \
    shift(1) == 0), 'unique_predicted_injuries_52_week'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries_52_week'] == 0) & (dataset_with_prediction['predicted_injuries_52_week']. \
    shift(1) == 1), 'unique_predicted_injuries_52_week'] = 0

In [235]:
# Getting the accumulated sum of predicted injuries by player

dataset_with_prediction['cum_predicted_injuries_1_week'] = dataset_with_prediction.groupby(by=['name'])['unique_predicted_injuries_1_week'].cumsum()
dataset_with_prediction['cum_predicted_injuries_4_week'] = dataset_with_prediction.groupby(by=['name'])['unique_predicted_injuries_4_week'].cumsum()
dataset_with_prediction['cum_predicted_injuries_12_week'] = dataset_with_prediction.groupby(by=['name'])['unique_predicted_injuries_12_week'].cumsum()
dataset_with_prediction['cum_predicted_injuries_26_week'] = dataset_with_prediction.groupby(by=['name'])['unique_predicted_injuries_26_week'].cumsum()
dataset_with_prediction['cum_predicted_injuries_52_week'] = dataset_with_prediction.groupby(by=['name'])['unique_predicted_injuries_52_week'].cumsum()

In [236]:
#Filtering the dataset for a subset of players

player_for_df_final = ['Lionel Messi', 'Cristiano Ronaldo', 'Neymar', 'Robert Lewandowski', 'Mohamed Salah', 'Kevin De Bruyne', 'Erling Haaland', \
'Harry Kane', 'Virgil van Dijk', 'Eden Hazard', 'Karim Benzema', 'Paul Pogba', 'Sergio Ramos', 'Jadon Sancho', 'Romelu Lukaku', \
'Joshua Kimmich', 'Antoine Griezmann', 'Toni Kroos', 'Jan Oblak', 'Manuel Neuer', 'Gareth Bale', 'Raheem Sterling', 'Frenkie de Jong', \
'Thiago Silva', 'Paulo Dybala']

small_dataset_with_prediction = pd.DataFrame([])

for n in player_for_df_final:
    small_dataset_with_prediction = small_dataset_with_prediction.append(dataset_with_prediction[dataset_with_prediction['name'] == n])

In [296]:
# Saving small_dataset_with_prediction to repository
small_dataset_with_prediction.to_parquet('dataframes_blog/small_dataset_with_prediction.parquet')

In [257]:
#Picking Player
picked_player_df = small_dataset_with_prediction[small_dataset_with_prediction['name'] == 'Neymar']

In [286]:
# Function to Forecast players injuries and plot a chart

def forecast_it(weeks, df1):
    df = df1.reset_index()
    final_index = df.index.max()
    pos = 'cum_injury_in_'+str(weeks)+'_week'
    for n in range(weeks,0,-1): 
        df.loc[final_index-n+1, pos] = ''
    df.loc[df[pos] == "", 'single_injury_prediction_in_'+str(weeks)+'_week'] = df[pos].shift(1) 
    df.loc[df['single_injury_prediction_in_'+str(weeks)+'_week'] == "", 'single_injury_prediction_in_'+str(weeks)+'_week'] = \
        df['unique_predicted_injuries_'+str(weeks)+'_week']
    df['cum_injury_prediction_in_'+str(weeks)+'_week'] = df['single_injury_prediction_in_'+str(weeks)+'_week'].cumsum()
    # df['cum_injury_prediction_in_'+str(weeks)+'_week'] = df['cum_injury_prediction_in_'+str(weeks)+'_week'].fillna('')
    # df.loc[df['cum_injury_in_'+str(weeks)+'_week'] != "", 'cum_injury_prediction_in_'+str(weeks)+'_week'] = df['cum_injury_in_'+str(weeks)+'_week']
    df.loc[df['cum_injury_prediction_in_'+str(weeks)+'_week'].isna(), 'cum_injury_prediction_in_'+str(weeks)+'_week'] = df['cum_injury_in_'+str(weeks)+'_week']

    df.loc[df['cum_injury_in_'+str(weeks)+'_week'] != "", 'type_for_'+str(weeks)] = 'Actual Injuries'
    df.loc[df['cum_injury_in_'+str(weeks)+'_week'] == "", 'type_for_'+str(weeks)] = 'Predicted Injuries'

    extra_point = df[df['type_for_'+str(weeks)] == 'Actual Injuries'].tail(1)
    extra_point['type_for_'+str(weeks)] = 'Predicted Injuries'
    df = pd.concat([df, extra_point]) #.reset_index(drop=True)

    df.loc[df['attacker'] == 1, 'position'] = 'attacker'
    df.loc[df['defender'] == 1, 'position'] = 'defender'
    df.loc[df['goalkeeper'] == 1, 'position'] = 'goalkeeper'
    df.loc[df['midfielder'] == 1, 'position'] = 'midfielder'

    df = df[['FBRefID', 'name', 'date', 'cum_week', 'position', 'Min_cum', 'cum_injury_prediction_in_'+str(weeks)+'_week', 'type_for_'+str(weeks)]]
    
    df.loc[df['cum_injury_prediction_in_'+str(weeks)+'_week'] == '', 'cum_injury_prediction_in_'+str(weeks)+'_week'] = None
    df['cum_injury_prediction_in_'+str(weeks)+'_week'] = df['cum_injury_prediction_in_'+str(weeks)+'_week'].ffill()
    
    
    return df

In [290]:
# Forecasting the different horizons for picked player

final_small_df_1_week = forecast_it(1, picked_player_df)
final_small_df_4_week = forecast_it(4, picked_player_df)
final_small_df_12_week = forecast_it(12, picked_player_df)
final_small_df_26_week = forecast_it(26, picked_player_df)
final_small_df_52_week = forecast_it(52, picked_player_df)

In [297]:
# Function to change name of columns of all dataframes

def transform_it(weeks, df):
    df = df.rename(columns={'cum_injury_prediction_in_'+str(weeks)+'_week':'predicted_cum_injuries', 'type_for_'+str(weeks):'label'})
    df['horizon'] = weeks
    return df

In [302]:
# Changing column names and concatenating dataframes

final_df_lst = [final_small_df_week, final_small_df_month, final_small_df_quarter, final_small_df_semester, final_small_df_year]
weeks_lst = [1, 4, 12, 26, 52]

final_player_df = pd.DataFrame(columns=list(transform_it(1,final_small_df_week).columns))
counter = 0

for df in final_df_lst:
    final_player_df = final_player_df.append(transform_it(weeks_lst[counter], df))
    counter += 1

In [303]:
#Plotting Chart
def plot_it(weeks, df):
    
    df = df[df['horizon'] == weeks]

    if weeks == 1:
        df_final = df[len(df)-25:].sort_index()
    elif weeks == 4:
        df_final = df[len(df)-25:].sort_index()
    elif weeks == 12:
        df_final = df[len(df)-25:].sort_index()
    else:
        df_final = df[len(df)-(weeks*2):].sort_index()

    domain_min = df_final['predicted_cum_injuries'].min()-5
    domain_max = df_final['predicted_cum_injuries'].max()+5

    chart = alt.Chart(df_final).mark_line().encode(x=alt.X('date', timeUnit='yearmonthdate'), \
        y=alt.Y('predicted_cum_injuries', scale=alt.Scale(domain=[domain_min, domain_max])), color=alt.Color('label'), \
        strokeDash=alt.condition(alt.datum.symbol == 'label', alt.value([5, 5]), alt.value([0]))). \
        properties(title = str(df_final['name'].iloc[0])+"'s Injury Prediction for "+str(weeks)+" weeks", width=800, height=300)

    return chart
    

In [310]:
plot_it(52, final_player_df)

alt.Chart(...)